In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import Add
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Reshape, GlobalAveragePooling2D, Activation,UpSampling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

from mediapipe.python.solutions import pose as mp_pose
import cv2
pose_tracker = mp_pose.Pose()

In [2]:
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN_GRAY_landmarks/'
valid_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID_GRAY_landmarks/'
test_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TEST_GRAY_landmarks/'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=32)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224, 224), batch_size=32)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=32, shuffle=False)

Found 10554 images belonging to 15 classes.
Found 2990 images belonging to 15 classes.
Found 1525 images belonging to 15 classes.


In [3]:
import os
import cv2
import numpy as np
from mediapipe.python.solutions import pose as mp_pose
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def flow_from_directory_with_landmarks(
    generator,
    directory,
    target_size=(224, 224),
    color_mode="rgb",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
):
    # Get the original flow_from_directory output
    iterator = generator.flow_from_directory(
        directory,
        target_size=target_size,
        color_mode=color_mode,
        classes=classes,
        class_mode=class_mode,
        batch_size=batch_size,
        shuffle=shuffle,
        seed=seed,
        save_to_dir=save_to_dir,
        save_prefix=save_prefix,
        save_format=save_format,
        follow_links=follow_links,
        subset=subset,
        interpolation=interpolation,
    )

    # Create empty lists to store image data and landmarks
    image_data = []
    landmarks = []

    # Load and preprocess each image with landmarks
    pose_tracker = mp_pose.Pose()
    for image_path in iterator.filepaths:
        # Read the image
        image = cv2.imread(image_path)

        # Extract landmarks using mediapipe.pose
        results = pose_tracker.process(image)
        pose_landmarks = results.pose_landmarks

        # Check if landmarks were detected
        if pose_landmarks is not None:
            # Extract landmark coordinates
            landmark_coordinates = [[lmk.x, lmk.y, 0.0] for lmk in pose_landmarks.landmark]
            landmarks.append(landmark_coordinates)
        else:
            # If landmarks were not detected, append zeros
            landmarks.append(np.zeros((33, 3)))

        # Preprocess and resize the image
        image = cv2.resize(image, target_size)
        image_data.append(image)

    # Convert the image data and landmarks to numpy arrays
    image_data = np.array(image_data)
    landmarks = np.array(landmarks)

    # Append zeros and reshape landmarks array
    landmarks = np.concatenate((landmarks, np.zeros((len(image_data), 50176 - 33, 3))), axis=1)
    landmarks = landmarks.reshape((len(image_data), 224, 224, 3))

    # Return the modified image data and labels
    return image_data, landmarks, iterator.labels


# Usage example
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN_GRAY_landmarks/'
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

# Generate batches of image data and landmarks from the directory
train_images, train_landmarks, train_labels = flow_from_directory_with_landmarks(
    train_batches,
    directory=train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Verify the shapes of the data
print(train_images.shape)       # Shape of train_images
print(train_landmarks.shape)    # Shape of train_landmarks
print(train_labels.shape)       # Shape of train_labels

# Convert training labels to categorical format
train_labels_encoded = tf.keras.utils.to_categorical(train_labels, num_classes=15)
print(train_labels_encoded.shape)

Found 10554 images belonging to 15 classes.


MemoryError: Unable to allocate 11.8 GiB for an array with shape (10554, 50176, 3) and data type float64

In [ ]:
# Modify the variable names and paths for validation data
val_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID_GRAY_landmarks/'
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

# Generate batches of image data and landmarks from the validation directory
val_images, val_landmarks, val_labels = flow_from_directory_with_landmarks(
    val_batches,
    directory=val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Verify the shapes of the validation data
print(val_images.shape)       # Shape of val_images
print(val_landmarks.shape)    # Shape of val_landmarks
print(val_labels.shape)       # Shape of val_labels

In [ ]:
valid_labels_encoded = tf.keras.utils.to_categorical(val_labels, num_classes=15)

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.applications.mobilenet import preprocess_input
from mediapipe.python.solutions import pose as mp_pose
import tensorflow as tf

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dropout, Dense, concatenate
from tensorflow.keras.models import Model

model1 = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Dropout(0.2)

])

# Build the model
model1.build((None, 224, 224, 3))
model1.summary()

In [ ]:
model2 = tf.keras.Sequential([
    hub.KerasLayer(
        'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5',
        trainable=True),
    tf.keras.layers.Dense(128, activation='relu'),
])

# Build the model
model2.build((None, 224, 224, 3))
model2.summary()

In [ ]:
combined = concatenate([model1.output,model2.output])
prediction = Dense(15, activation='softmax')(combined)

fin_model = Model(inputs=[model1.input,model2.input], outputs=prediction)
fin_model.summary()

In [ ]:
fin_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
epochs = 10
batch_size = 32


fin_model.fit(
    [train_landmarks,train_images],  
    train_labels_encoded,                     
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([val_landmarks,val_images], valid_labels_encoded),
    verbose = 1
)

In [19]:
train_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/TRAIN_landmarks/'
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

# Generate batches of image data and landmarks from the directory
train_images, train_landmarks, train_labels = flow_from_directory_with_landmarks(
    train_batches,
    directory=train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Verify the shapes of the data
print(train_images.shape)       # Shape of train_images
print(train_landmarks.shape)    # Shape of train_landmarks
print(train_labels.shape)       # Shape of train_labels

# Convert training labels to categorical format
train_labels_encoded = tf.keras.utils.to_categorical(train_labels, num_classes=15)
print(train_labels_encoded.shape)

Found 10554 images belonging to 15 classes.
(10554, 224, 224, 3)
(10554, 32, 32, 3)
(10554,)
(10554, 15)


In [20]:
# Modify the variable names and paths for validation data
val_path = 'C:/Users/Prasanna P M/Human Project/ML_aug/Human Action Recognition/VALID_landmarks/'
val_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input)

# Generate batches of image data and landmarks from the validation directory
val_images, val_landmarks, val_labels = flow_from_directory_with_landmarks(
    val_batches,
    directory=val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Verify the shapes of the validation data
print(val_images.shape)       # Shape of val_images
print(val_landmarks.shape)    # Shape of val_landmarks
print(val_labels.shape)       # Shape of val_labels

Found 2990 images belonging to 15 classes.
(2990, 224, 224, 3)
(2990, 32, 32, 3)
(2990,)


In [21]:
valid_labels_encoded = tf.keras.utils.to_categorical(val_labels, num_classes=15)

In [22]:
fin_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

In [ ]:
epochs = 10
batch_size = 32


fin_model.fit(
    [train_landmarks,train_images],  
    train_labels_encoded,                     
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([val_landmarks,val_images], valid_labels_encoded),
    verbose = 1
)

Epoch 1/10
330/330 [==============================] - 1705s 5s/step - loss: 0.5749 - accuracy: 0.8954 - val_loss: 1.7146 - val_accuracy: 0.6445
Epoch 2/10
330/330 [==============================] - 1710s 5s/step - loss: 0.3562 - accuracy: 0.9686 - val_loss: 1.7966 - val_accuracy: 0.6629
Epoch 3/10
  1/330 [..............................] - ETA: 26:49 - loss: 0.2753 - accuracy: 1.0000